In [1]:
import torch
import random

import torch.nn as nn

/Users/luan/Developer/bow-text-classifier/.venv/lib/python3.11/site-packages/torch/nn/modules/transformer.py:20: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  device: torch.device = torch.device(torch._C._get_default_device()),  # torch.device('cpu'),


In [2]:
from bow_text_classifier.data import (
    get_dataset_filepath,
    _parse_data,
    _create_dict,
    _create_tensors,
)

# with open(get_dataset_filepath("dev"), "r") as f:
#     dev_data = _parse_data(f.read())

with open(get_dataset_filepath("train"), "r") as f:
    train_data = _parse_data(f.read())

with open(get_dataset_filepath("test"), "r") as f:
    test_data = _parse_data(f.read())


word_to_index, tag_to_index = _create_dict(train_data)

word_to_index, tag_to_index = _create_dict(
    test_data, word_to_index, tag_to_index, check_unk=True
)

number_of_words = len(word_to_index)
number_of_tags = len(tag_to_index)

train_data = list(_create_tensors(train_data, word_to_index, tag_to_index))
test_data = list(_create_tensors(test_data, word_to_index, tag_to_index))

number_of_words = len(word_to_index)
number_of_tags = len(tag_to_index)

In [3]:
from bow_text_classifier.nn import BoW, sentence_to_tensor

device = "cuda" if torch.cuda.is_available() else "cpu"


type = torch.cuda.LongTensor if torch.cuda.is_available() else torch.LongTensor
out = sentence_to_tensor("i love dogs", word_to_index).type(type)
test_model = BoW(number_of_words, number_of_tags).to(device)

test_model(out)

tensor([[ 0.0099, -0.0141,  0.0240, -0.0026, -0.0072]],
       grad_fn=<ViewBackward0>)

In [4]:
from bow_text_classifier.nn import train_bow

# train and test the BoW model
model = BoW(number_of_words, number_of_tags).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
type = torch.LongTensor

if torch.cuda.is_available():
    model.to(device)
    type = torch.cuda.LongTensor


# call the train_bow function
train_bow(model, optimizer, criterion, train_data, test_data, type)

ITER: 1 | train loss/sent: 1.4739 | train accuracy: 0.3708 | test accuracy: 0.3977
ITER: 2 | train loss/sent: 1.1212 | train accuracy: 0.6058 | test accuracy: 0.4113
ITER: 3 | train loss/sent: 0.9114 | train accuracy: 0.7140 | test accuracy: 0.4104
ITER: 4 | train loss/sent: 0.7689 | train accuracy: 0.7690 | test accuracy: 0.4109
ITER: 5 | train loss/sent: 0.6637 | train accuracy: 0.8059 | test accuracy: 0.4127
ITER: 6 | train loss/sent: 0.5822 | train accuracy: 0.8315 | test accuracy: 0.4032
ITER: 7 | train loss/sent: 0.5165 | train accuracy: 0.8529 | test accuracy: 0.4081
ITER: 8 | train loss/sent: 0.4637 | train accuracy: 0.8737 | test accuracy: 0.3955
ITER: 9 | train loss/sent: 0.4190 | train accuracy: 0.8810 | test accuracy: 0.3914
ITER: 10 | train loss/sent: 0.3812 | train accuracy: 0.8944 | test accuracy: 0.3955


In [5]:
def perform_inference(model, sentence, word_to_index, tag_to_index, device):
    """
    Perform inference on the trained BoW model.

    Args:
        model (torch.nn.Module): The trained BoW model.
        sentence (str): The input sentence for inference.
        word_to_index (dict): A dictionary mapping words to their indices.
        tag_to_index (dict): A dictionary mapping tags to their indices.
        device (str): "cuda" or "cpu" based on availability.

    Returns:
        str: The predicted class/tag for the input sentence.
    """
    # Preprocess the input sentence to match the model's input format
    sentence_tensor = sentence_to_tensor(sentence, word_to_index)

    # Move the input tensor to the same device as the model
    sentence_tensor = sentence_tensor.to(device)

    # Make sure the model is in evaluation mode and on the correct device
    model.eval()
    model.to(device)

    # Perform inference
    with torch.no_grad():
        output = model(sentence_tensor)

    # Move the output tensor to CPU if it's on CUDA
    if device == "cuda":
        output = output.cpu()

    # Convert the model's output to a predicted class/tag
    predicted_class = torch.argmax(output).item()

    # Reverse lookup to get the tag corresponding to the predicted class
    for tag, index in tag_to_index.items():
        if index == predicted_class:
            return tag

    # Return an error message if the tag is not found
    return "Tag not found"


# Example usage:
# Load your trained model (model) and dictionaries (word_to_index and tag_to_index) here
# model = load_trained_model()
# word_to_index, tag_to_index = load_dictionaries()

# Determine the device based on availability
device = "cuda" if torch.cuda.is_available() else "cpu"

sample_sentence = "I love programming"
predicted_tag = perform_inference(
    model, sample_sentence, word_to_index, tag_to_index, device
)
print(f"Predicted Tag: {predicted_tag}")

False
Predicted Tag: 4
